In [21]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import joblib

In [29]:
def prepare_data():
  images = []
  labels = []
  categories = []

  data_dir = 'D:\PROJECTS\IMAGE CLASSIFIER\Data'
  for folder in os.listdir(data_dir):
    categories.append(folder)

#   counter = 0
  for i in categories:
    path = os.path.join(data_dir, i)
    for img in os.listdir(path):
      image = cv2.imread(os.path.join(path, img))
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = cv2.resize(image, (224, 224))
      images.append(image)
      labels.append(i)
#       counter += 1
#       if counter == 200:
#         counter = 0
#         break
    print('folder completed!')
  images = np.array(images)
  labels = np.array(labels)
  print(categories)
  return images, labels

In [30]:
images, labels = prepare_data()

folder completed!
folder completed!
folder completed!
folder completed!
['cloudy', 'Rain', 'shine', 'sunrise']


In [31]:
def extract_features(images):
  model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  vgg16_features = model.predict(images)
  print(model.summary())
  return vgg16_features

features = extract_features(images)
print(features.shape)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [32]:
examples = features.shape[0]
_features = 1
for ele in features.shape[1:]:
  _features *= ele
features = features.flatten().reshape((examples, _features))
print(features.shape)

(1120, 25088)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, stratify=labels, random_state=1)

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(),
#     DecisionTreeClassifier(),
#     RandomForestClassifier(),
#     AdaBoostClassifier(),
#     GradientBoostingClassifier(),
#     XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=4)
# ]

# for model in classifiers:
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     print(f'The Classification Report for {model} is:')
#     print('\n')
#     print(classification_report(y_test, y_pred))
#     print('\n')
#     print(confusion_matrix(y_test, y_pred))
#     print('\n')

model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f'The Classification Report for {model} is:')
print('\n')
print(classification_report(y_test, y_pred))
print('\n')
print(confusion_matrix(y_test, y_pred))
print('\n')

The Classification Report for SVC() is:


              precision    recall  f1-score   support

        Rain       1.00      0.95      0.98        63
      cloudy       0.96      0.94      0.95        90
       shine       0.91      0.97      0.94        76
     sunrise       0.97      0.96      0.97       107

    accuracy                           0.96       336
   macro avg       0.96      0.96      0.96       336
weighted avg       0.96      0.96      0.96       336



[[ 60   1   1   1]
 [  0  85   3   2]
 [  0   2  74   0]
 [  0   1   3 103]]




In [39]:
joblib.dump(model, 'D:/PROJECTS/IMAGE CLASSIFIER/model/model_joblib')

['D:/PROJECTS/IMAGE CLASSIFIER/model/model_joblib']

In [22]:
mj = joblib.load('D:\PROJECTS\IMAGE CLASSIFIER\model\model_joblib')

In [23]:
image = cv2.imread('D:/PROJECTS/IMAGE CLASSIFIER/Data/sunrise/sunrise15.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

print(image.shape)
def extract_features(image):
  model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  vgg16_features = model.predict(image)
  print(model.summary())
  return vgg16_features

image = image.reshape(-1, 224, 224, 3)
features = extract_features(image)
features = features.reshape(1, -1)
prediction = mj.predict(features)
prediction

(224, 224, 3)
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128

array(['sunrise'], dtype='<U7')